# kaggle study 32일차(zillow)
코드출처 : https://www.kaggle.com/aharless/xgboost-lightgbm-and-ols-and-nn

In [1]:
# Parameters
FUDGE_FACTOR = 1.1200 # 예측에 이를 곱합니다.

XGB_WEIGHT = 0.6200
BASELINE_WEIGHT = 0.0100
OLS_WEIGHT = 0.0620
NN_WEIGHT = 0.0800

XGB1_WEIGHT = 0.8000 # 두 XGB 모델의 조합에서 첫 번째의 무게입니다.

BASELINE_PRED = 0.0115  # Oleg당 교육 데이터 평균에 기초합니다.

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

### READ IN RAW DATA

In [3]:
prop = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/properties_2016.csv')
train =  pd.read_csv("C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/train_2016_v2.csv")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


##  LightGBM
이 섹션은 원래 SIDHARTH의 스크립트에서 파생된 것입니다.  
https://www.kaggle.com/sidharthkumar/trying-lightgbm  
Yuching Xue가 포크 및 튜닝한 것입니다.  
https://www.kaggle.com/yuqingxue/lightgbm-85-97  
그리고 필자가 업데이트를 진행했습니다.(앤디 할리스):  
링크 : https://www.kaggle.com/aharless/lightgbm-with-outliers-remaining  
그 이후 많은 추가 변경 사항이 발생했습니다.  

### PROCESS DATA FOR LIGHTGBM

In [4]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')
df_train.fillna(df_train.median(),inplace = True)

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
#x_train['Ratio_1'] = x_train['taxvaluedollarcnt']/x_train['taxamount']
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)


train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)

(90275, 53) (90275,)


### RUN LIGHTGBM

In [5]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    # feature_fraction (small values => use very different submodels)
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

np.random.seed(0)
random.seed(0)

#### Fitting LightGBM model

In [6]:
clf = lgb.train(params, d_train, 430)

del d_train; gc.collect()
del x_train; gc.collect()

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003890 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0

#### Prepare for LightGBM prediction

In [7]:
sample = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow//sample_submission.csv')
sample['parcelid'] = sample['ParcelId']

#### Merge with property data

In [8]:
df_test = sample.merge(prop, on='parcelid', how='left')
del sample, prop; gc.collect()
x_test = df_test[train_columns]
print("   ...")
del df_test; gc.collect()

   ...


0

#### Preparing x_test

In [9]:
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
    
x_test = x_test.values.astype(np.float32, copy=False)

#### Start LightGBM prediction

In [10]:
p_test = clf.predict(x_test)

del x_test; gc.collect()

print( "\nUnadjusted LightGBM predictions:" )
print( pd.DataFrame(p_test).head() )


Unadjusted LightGBM predictions:
          0
0  0.031132
1  0.033375
2  0.010257
3  0.008651
4  0.009660


###  XGBoost

이 섹션은 원래 Infinite Wing의 스크립트에서 파생된 것입니다.  
링크 : https://www.kaggle.com/infinitewing/xgboost-without-outliers-lb-0-06463  
이 스레드에서 영감을 받았습니다.  
링크 : https://www.kaggle.com/c/zillow-prize-1/discussion/33710  
하지만 그 이후로 코드는 많은 변화를 겪었습니다.  

#### RE-READ PROPERTIES FILE
복사본을 보관하려고 했지만 프로그램이 중단되었습니다.

In [11]:
properties = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow//properties_2016.csv')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### PROCESS DATA FOR XGBOOST

In [12]:
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

train_df = train.merge(properties, how='left', on='parcelid')
x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)
# shape        
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

Shape train: (90275, 57)
Shape test: (2985217, 57)


#### drop ouliers

In [13]:
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
x_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

After removing outliers:
Shape train: (88528, 57)
Shape test: (2985217, 57)


#### RUN XGBOOST

In [14]:
# xgboost params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 250
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xgb_pred1 = model.predict(dtest)

print( "\nFirst XGBoost predictions:" )
print( pd.DataFrame(xgb_pred1).head() )

num_boost_rounds=250

Training XGBoost ...
[14:20:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:20:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



Predicting with XGBoost ...

First XGBoost predictions:
          0
0 -0.030616
1 -0.028188
2  0.026397
3  0.063728
4  0.004398


#### RUN XGBOOST AGAIN
##### Setting up data for XGBoost

In [15]:
# xgboost params
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)
xgb_pred2 = model.predict(dtest)

print( "\nSecond XGBoost predictions:" )
print( pd.DataFrame(xgb_pred2).head() )

num_boost_rounds=150
[14:21:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:21:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



Second XGBoost predictions:
          0
0 -0.091150
1 -0.034722
2  0.015816
3  0.075518
4  0.029908


#### COMBINE XGBOOST RESULTS

In [16]:
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2

print( "\nCombined XGBoost predictions:" )
print( pd.DataFrame(xgb_pred).head() )

del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2 
gc.collect()


Combined XGBoost predictions:
          0
0 -0.042723
1 -0.029495
2  0.024281
3  0.066086
4  0.009500


90

###  Neural Network

이 스크립트에서 복사한 신경망입니다.  
링크 : https://www.kaggle.com/aharless/keras-neural-network-lb-06492 (버전 20)  
이 노트북의 골격 위에 작성되었습니다.  
링크 : https://www.kaggle.com/prasunmishra/ann-using-keras  

#### Read in data for neural network

In [17]:
train = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/train_2016_v2.csv')
prop = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/properties_2016.csv')
sample = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow//sample_submission.csv')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Fitting Label Encoder on properties

In [18]:
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

#### Creating training set

In [19]:
df_train = train.merge(prop, how='left', on='parcelid')

df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train["transactiondate_year"] = df_train["transactiondate"].dt.year
df_train["transactiondate_month"] = df_train["transactiondate"].dt.month
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
df_train["transactiondate"] = df_train["transactiondate"].dt.day

#### Filling NA/NaN values

In [20]:
df_train.fillna(-1.0)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate_year,transactiondate_month,transactiondate_quarter
0,11016594,0.0276,1,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,...,360170.0,2015.0,237416.0,6735.88,0,-1.0,6.037107e+13,2016,1,1
1,14366692,-0.1684,1,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,...,585529.0,2015.0,239071.0,10153.02,0,-1.0,-1.000000e+00,2016,1,1
2,12098116,-0.0040,1,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,...,119906.0,2015.0,57912.0,11484.48,0,-1.0,6.037464e+13,2016,1,1
3,12643413,0.0218,2,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,244880.0,2015.0,73362.0,3048.74,0,-1.0,6.037296e+13,2016,1,1
4,14432541,-0.0050,2,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,...,434551.0,2015.0,264977.0,5488.96,0,-1.0,6.059042e+13,2016,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,30,1.0,-1.0,-1.0,1.0,1.0,-1.0,4.0,...,191000.0,2015.0,147200.0,2495.24,0,-1.0,6.037132e+13,2016,12,4
90271,12046695,0.0070,30,-1.0,-1.0,-1.0,3.0,3.0,-1.0,4.0,...,161111.0,2015.0,43218.0,1886.54,0,-1.0,6.037301e+13,2016,12,4
90272,12995401,-0.2679,30,-1.0,-1.0,-1.0,2.0,4.0,-1.0,7.0,...,38096.0,2015.0,16088.0,1925.70,1,14.0,6.037433e+13,2016,12,4
90273,11402105,0.0602,30,-1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,165869.0,2015.0,32878.0,2285.57,0,-1.0,6.037601e+13,2016,12,4


#### Creating x_train and y_train from df_train

In [21]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode','fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train["logerror"]

y_mean = np.mean(y_train)
print(x_train.shape, y_train.shape)
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

(90275, 56) (90275,)


#### Creating df_test

In [22]:
sample['parcelid'] = sample['ParcelId']

#### Merging Sample with property data

In [23]:
df_test = sample.merge(prop, on='parcelid', how='left')

df_test["transactiondate"] = pd.to_datetime('2016-11-15')  # placeholder value for preliminary version
df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
df_test["transactiondate"] = df_test["transactiondate"].dt.day     
x_test = df_test[train_columns]

print('Shape of x_test:', x_test.shape)
print("Preparing x_test...")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

Shape of x_test: (2985217, 56)
Preparing x_test...


#### Preprocessing

In [24]:
imputer= SimpleImputer()
imputer.fit(x_train.iloc[:, :])
x_train = imputer.transform(x_train.iloc[:, :])
imputer.fit(x_test.iloc[:, :])
x_test = imputer.transform(x_test.iloc[:, :])

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

len_x=int(x_train.shape[1])
print("len_x is:",len_x)

len_x is: 56


#### Setting up neural network model

In [25]:
nn = Sequential()
nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units = 26, kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(1, kernel_initializer='normal'))
nn.compile(loss='mae', optimizer=Adam(lr=4e-3, decay=1e-4))

#### Fitting neural network model

In [26]:
nn.fit(np.array(x_train), np.array(y_train), batch_size = 32, epochs = 70, verbose=2)

Epoch 1/70
2822/2822 - 10s - loss: 0.0713
Epoch 2/70
2822/2822 - 7s - loss: 0.0682
Epoch 3/70
2822/2822 - 7s - loss: 0.0682
Epoch 4/70
2822/2822 - 7s - loss: 0.0681
Epoch 5/70
2822/2822 - 9s - loss: 0.0680
Epoch 6/70
2822/2822 - 9s - loss: 0.0680
Epoch 7/70
2822/2822 - 8s - loss: 0.0679
Epoch 8/70
2822/2822 - 6s - loss: 0.0678
Epoch 9/70
2822/2822 - 6s - loss: 0.0678
Epoch 10/70
2822/2822 - 6s - loss: 0.0678
Epoch 11/70
2822/2822 - 6s - loss: 0.0677
Epoch 12/70
2822/2822 - 6s - loss: 0.0677
Epoch 13/70
2822/2822 - 6s - loss: 0.0677
Epoch 14/70
2822/2822 - 6s - loss: 0.0677
Epoch 15/70
2822/2822 - 6s - loss: 0.0676
Epoch 16/70
2822/2822 - 7s - loss: 0.0677
Epoch 17/70
2822/2822 - 7s - loss: 0.0676
Epoch 18/70
2822/2822 - 8s - loss: 0.0676
Epoch 19/70
2822/2822 - 7s - loss: 0.0676
Epoch 20/70
2822/2822 - 8s - loss: 0.0676
Epoch 21/70
2822/2822 - 9s - loss: 0.0676
Epoch 22/70
2822/2822 - 8s - loss: 0.0675
Epoch 23/70
2822/2822 - 9s - loss: 0.0675
Epoch 24/70
2822/2822 - 8s - loss: 0.0675


#### Predicting with neural network model

In [27]:
y_pred_ann = nn.predict(x_test)

print( "\nPreparing results for write..." )
nn_pred = y_pred_ann.flatten()
print( "Type of nn_pred is ", type(nn_pred) )
print( "Shape of nn_pred is ", nn_pred.shape )

print( "\nNeural Network predictions:" )
print( pd.DataFrame(nn_pred).head() )



Preparing results for write...
Type of nn_pred is  <class 'numpy.ndarray'>
Shape of nn_pred is  (2985217,)

Neural Network predictions:
          0
0 -0.015185
1 -0.020522
2  0.703887
3  0.165104
4  0.164292


In [28]:
# Cleanup
del train
del prop
del sample
del x_train
del x_test
del df_train
del df_test
del y_pred_ann
gc.collect()

2012

#### OLS 

이 섹션은 1owl의 노트북에서 파생되었습니다.  
링크 : https://www.kaggle.com/the1owl/primer-for-the-zillow-pred-approach  
저(Andy Harless)가 업데이트하고 스크립트로 만들었습니다.  
링크 : https://www.kaggle.com/aharless/updated-script-version-of-the1owl-s-basic-ols 

In [29]:
np.random.seed(17)
random.seed(17)

#### Processing data for OLS 

In [30]:
train = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/train_2016_v2.csv')
properties = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/properties_2016.csv')
submission = pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow//sample_submission.csv')
print(len(train),len(properties),len(submission))

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


90275 2985217 2985217


In [31]:
def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

train = pd.merge(train, properties, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(submission, properties, how='left', left_on='ParcelId', right_on='parcelid')
properties = [] 

exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

KeyError: 'transactiondate'

#### Fitting OLS

In [ ]:
reg = LinearRegression(n_jobs=-1)
reg.fit(train, y); print('fit...')
print(MAE(y, reg.predict(train)))
train = [];  y = [] #memory

test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']

####  Combine and Save

In [ ]:
print( "\nCombining XGBoost, LightGBM, NN, and baseline predicitons ..." )
lgb_weight = 1 - XGB_WEIGHT - BASELINE_WEIGHT - NN_WEIGHT - OLS_WEIGHT 
lgb_weight0 = lgb_weight / (1 - OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT / (1 - OLS_WEIGHT)
baseline_weight0 =  BASELINE_WEIGHT / (1 - OLS_WEIGHT)
nn_weight0 = NN_WEIGHT / (1 - OLS_WEIGHT)
pred0 = 0
pred0 += xgb_weight0*xgb_pred
pred0 += baseline_weight0*BASELINE_PRED
pred0 += lgb_weight0*p_test
pred0 += nn_weight0*nn_pred

print( "\nCombined XGB/LGB/NN/baseline predictions:" )
print( pd.DataFrame(pred0).head() )

print( "\nPredicting with OLS and combining with XGB/LGB/NN/baseline predicitons: ..." )
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = FUDGE_FACTOR * ( OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0 )
    submission[test_columns[i]] = [float(format(x, '.4f')) for x in pred]
    print('predict...', i)

print( "\nCombined XGB/LGB/NN/baseline/OLS predictions:" )
print( submission.head() )


#### WRITE THE RESULTS

In [ ]:
from datetime import datetime

print( "\nWriting results to disk ..." )
submission.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)